In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

In [2]:
train_dataset = dataset["train"].shuffle(seed=42).select([i for i in list(range(1000))])
validation_dataset = dataset["test"].shuffle(seed=42).select([i for i in list(range(100))])
test_dataset = dataset["test"].shuffle(seed=42).select([i for i in list(range(50))])


Loading cached shuffled indices for dataset at /Users/yangwoolee/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-a0e621c27d9b360e.arrow
Loading cached shuffled indices for dataset at /Users/yangwoolee/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-61e0da4d9cd46a2c.arrow
Loading cached shuffled indices for dataset at /Users/yangwoolee/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-61e0da4d9cd46a2c.arrow


In [5]:
train_dataset.to_csv('data/train.csv',index=False)
validation_dataset.to_csv('data/validation.csv',index=False)
test_dataset.to_csv('data/test.csv',index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

36382

In [ ]:
from transformers import DistilBertForTokenClassification, DistilBertTokenizer
from datasets import Dataset

model = DistilBertForTokenClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(train_dataset['label'])))
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


train_dataset = Dataset.from_csv('data/train.csv')
validation_dataset = Dataset.from_csv('data/validation.csv')

def tokenize_function(item) :
    return tokenizer(item['text'], padding='max_length', max_length=128,truncation=True)

train = train_dataset.map(tokenize_function)
validation = validation_dataset.map(tokenize_function)


In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(train_dataset['label'])))
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [71]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    print('metric')
    return metric.compute(predictions=predictions, references=labels)

In [74]:
from transformers import Trainer,TrainingArguments
import os

os.environ['DISABLE_MLFLOW_INTEGRATION'] = "True"


tra_arg = TrainingArguments(
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_steps=10,
    num_train_epochs=1,
    output_dir='test',
    evaluation_strategy='step'
)


trainer = Trainer(
    model=model,
    args=tra_arg,
    train_dataset=train,
    eval_dataset=validation,
    compute_metrics=compute_metrics
)

trainer.train()

ValueError: step is not a valid IntervalStrategy, please select one of ['no', 'steps', 'epoch']